In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession \
    .builder \
    .appName("File Streaming Demo") \
    .master("local[3]") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 8)

In [2]:
# df_raw = spark.read.format("text").load("./dataset/examples.txt")

In [3]:
df_raw = spark.readStream.format("socket").option("host","localhost").option("port", "9999").load()

In [4]:
df_words = df_raw.withColumn("words", F.split("value", " "))
df_explode = df_words.withColumn("word", F.explode("words")).drop("value", "words")
df_agg = df_explode.groupBy("word").count()

In [6]:
# df_agg.writeStream.format("console").outputMode("update").start().awaitTermination()
stream = (
    df_agg
    .writeStream
    .format("console")
    .outputMode("complete")
    .start()
    )


stream.awaitTermination()

AnalysisException: Continuous processing does not support Generate operations.;
Generate explode(words#2), false, [word#6]
+- Project [value#0, split(value#0,  , -1) AS words#2]
   +- StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.TextSocketSourceProvider@7b41e67a, socket, org.apache.spark.sql.execution.streaming.sources.TextSocketTable@342e8ff6, [host=localhost, port=9999], [value#0]
